In [66]:
import pandas as pd
import numpy as np
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import category_encoders as ce
import pickle
from sklearn.svm import SVC
warnings.simplefilter(action='ignore', category=FutureWarning)

In [67]:
train = pd.read_csv("../data/sample_train.txt", delimiter="\t")
test = pd.read_csv("../data/test_id.txt", delimiter="\t")

In [68]:
df = pd.concat([train, test], axis=0)

In [69]:
risk = pd.read_csv("../features/risk/risk.csv")
df = df.merge(risk, on="id", how="left")

In [70]:
symbol = pd.read_csv("../features/symbol/symbol.csv")
df = df.merge(symbol, on="id", how="left")

In [71]:
# dp_emb = pd.read_csv("../features/graph/graph_filtered_64.emb")
# df = df.merge(dp_emb, on="id", how="left")

In [72]:
dp_emb = pd.read_csv("../features/graph/deepwalk_128_filtered.emb")
df = df.merge(dp_emb, on="id", how="left")

In [73]:
dp_emb.shape

(28442, 129)

In [74]:
# nv_emb = pd.read_csv("../data/edge/node2vec_64.emb")
# df = df.merge(nv_emb, on="id", how="left")

In [75]:
graph_info = pd.read_csv("../features/graph/graph_info.csv")
df = df.merge(graph_info, on="id", how="left")

In [76]:
# app_pca = pd.read_csv("../data/app/app_pca_16.csv")
# df = df.merge(app_pca, on="id", how="left")

app_lda = pd.read_csv("../features/app/app_lda_16.csv")
df = df.merge(app_lda, on="id", how="left")

# app_lda = pd.read_csv("../data1/app/app_lda_16.csv")
# df = df.merge(app_lda, on="id", how="left")

# app_nmf = pd.read_csv("../data/app/app_nmf_16.csv")
# df = df.merge(app_nmf, on="id", how="left")

In [77]:
# app_info = pd.read_csv("../features/app/app_info.csv")
# df = df.merge(app_info, on="id", how="left")

app_graph

In [78]:
# to_pca = pd.read_csv("../features/app_graph/to_app_pca_mean.csv")
# from_pca = pd.read_csv("../features/app_graph/from_app_pca_mean.csv")

# df = df.merge(to_pca, on="id", how="left")
# df = df.merge(from_pca, on="id", how="left")

to_pca = pd.read_csv("../features/app_graph/to_app_pca_num_mean.csv")
from_pca = pd.read_csv("../features/app_graph/from_app_pca_num_mean.csv")

df = df.merge(to_pca, on="id", how="left")
df = df.merge(from_pca, on="id", how="left")

to_pca = pd.read_csv("../features/app_graph/to_app_pca_weight_mean.csv")
from_pca = pd.read_csv("../features/app_graph/from_app_pca_weight_mean.csv")

df = df.merge(to_pca, on="id", how="left")
df = df.merge(from_pca, on="id", how="left")

In [79]:
# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_num_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_num_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

# to_pca_nmf = pd.read_csv("../features/app_graph/to_app_nmf_weight_mean.csv")
# from_pca_nmf = pd.read_csv("../features/app_graph/from_app_nmf_weight_mean.csv")

# df = df.merge(to_pca_nmf, on="id", how="left")
# df = df.merge(from_pca_nmf, on="id", how="left")

In [80]:
# to_lda = pd.read_csv("../features/app_graph/to_app_lda_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

# to_lda = pd.read_csv("../features/app_graph/to_app_lda_num_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_num_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

# to_lda = pd.read_csv("../features/app_graph/to_app_lda_weight_mean.csv")
# from_lda = pd.read_csv("../features/app_graph/from_app_lda_weight_mean.csv")

# df = df.merge(to_lda, on="id", how="left")
# df = df.merge(from_lda, on="id", how="left")

risk_graph

In [81]:
# to_risk = pd.read_csv("../features/risk_graph/to_risk_mean.csv")
# from_risk = pd.read_csv("../features/risk_graph/from_risk_mean.csv")

# df = df.merge(to_risk, on="id", how="left")
# df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_num_mean.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_num_mean.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_weight_mean.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_weight_mean.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

# to_risk = pd.read_csv("../features/risk_graph/to_risk_sum.csv")
# from_risk = pd.read_csv("../features/risk_graph/from_risk_sum.csv")

# df = df.merge(to_risk, on="id", how="left")
# df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_num_sum.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_num_sum.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

to_risk = pd.read_csv("../features/risk_graph/to_risk_weight_sum.csv")
from_risk = pd.read_csv("../features/risk_graph/from_risk_weight_sum.csv")

df = df.merge(to_risk, on="id", how="left")
df = df.merge(from_risk, on="id", how="left")

symbol_graph

In [82]:
# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_sum.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_sum.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_num_sum.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_num_sum.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

# to_cat_weight = pd.read_csv("../features/symbol_graph/to_symbol_weight_sum.csv")
# from_cat_weight = pd.read_csv("../features/symbol_graph/from_symbol_weight_sum.csv")

# df = df.merge(to_cat_weight, on="id", how="left")
# df = df.merge(from_cat_weight, on="id", how="left")

# to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_mean.csv")
# from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_mean.csv")

# df = df.merge(to_cat_num, on="id", how="left")
# df = df.merge(from_cat_num, on="id", how="left")

to_cat_num = pd.read_csv("../features/symbol_graph/to_symbol_num_mean.csv")
from_cat_num = pd.read_csv("../features/symbol_graph/from_symbol_num_mean.csv")

df = df.merge(to_cat_num, on="id", how="left")
df = df.merge(from_cat_num, on="id", how="left")

to_cat_weight = pd.read_csv("../features/symbol_graph/to_symbol_weight_mean.csv")
from_cat_weight = pd.read_csv("../features/symbol_graph/from_symbol_weight_mean.csv")

df = df.merge(to_cat_weight, on="id", how="left")
df = df.merge(from_cat_weight, on="id", how="left")

knn

In [83]:
# dp_knn = pd.read_csv("../features/knn/dp_knn_3.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_10.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_30.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_100.csv")
# df = df.merge(dp_knn, on="id", how="left")

# dp_knn = pd.read_csv("../features/knn/dp_knn_300.csv")
# df = df.merge(dp_knn, on="id", how="left")

中心性

In [84]:
pr = pd.read_csv("../features/graph/pagerank.csv")
df = df.merge(pr, on="id", how="left")

In [85]:
for d in ["to", "from"]:
    for w in ['', '_num', '_weight']:
        for f in ['sum', 'mean']:
            a = pd.read_csv("../features/graph/%s_pagerank%s_%s.csv" % (d, w, f))
            df = df.merge(a, on="id", how="left")

In [86]:
hits = pd.read_csv("../features/graph/hits.csv")
df = df.merge(hits, on="id", how="left")

for d in ["to", "from"]:
    for w in ['', '_num', '_weight']:
        for f in ['sum', 'mean']:
            a = pd.read_csv("../features/graph/%s_hits%s_%s.csv" % (d, w, f))
            df = df.merge(a, on="id", how="left")

In [87]:
hits = pd.read_csv("../features/graph/dc.csv")
df = df.merge(hits, on="id", how="left")
for d in ["to", "from"]:
    for w in ['', '_num', '_weight']:
        for f in ['sum', 'mean']:
            a = pd.read_csv("../features/graph/%s_dc%s_%s.csv" % (d, w, f))
            df = df.merge(a, on="id", how="left")

二度联系人

In [88]:
# 联系人个数
for d1 in ["from", "to"]:
    for d2 in ["from", "to"]:
        for t in ["", "_num", "_weight"]:
            for f in ["sum", "mean"]:
                a = pd.read_csv("../features/graph/%s_d2_%s%s_%s.csv" % (d1, d2, t, f))
                df = df.merge(a, on="id", how="left")

In [89]:
# 二度联系人PageRank值
for d1 in ["from", "to"]:
    for d2 in ["from", "to"]:
        for t in ["num", "weight"]:
            for f in ["sum", "mean"]:
                a = pd.read_csv("../features/graph/%s_pg_%s_%s_%s_%s_%s.csv" % (d1, d2, t, f, t, f))
                df = df.merge(a, on="id", how="left")

In [90]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

In [91]:
def  process_feature(train_x, valid_x, test_df):
    result = []
    drop_cols = ['id','label']
    for df in [train_x, valid_x, test_df]:
        result.append(df.drop(drop_cols, axis=1))
    return result 

In [95]:
import os
def cv(df, num_folds, param, model_dir, stratified=True, debug=False):
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    train_df = df[df.label.notnull()]
    test_df = df[df.label.isnull()]
    seed = 178
    if "seed" in param: 
        seed = param["seed"]
        del param['seed']
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
    oof_preds = np.zeros(train_df.shape[0])
    all_test_preds = []    
    feature_importance_df = pd.DataFrame()
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, train_df['label'])):
        train_x, train_y = train_df.iloc[train_idx], train_df['label'].iloc[train_idx]
        valid_x, valid_y = train_df.iloc[valid_idx], train_df['label'].iloc[valid_idx]
        fold_preds = test_df[["id"]]
        
        train_x, valid_x, test = process_feature(train_x, valid_x, test_df)
        if n_fold == 0:
            print(train_x.shape, valid_x.shape, test.shape)
        
        train_data = lgb.Dataset(train_x, label=train_y)
        validation_data = lgb.Dataset(valid_x, label=valid_y)

        clf=lgb.train(params,
                      train_data,
                      num_boost_round=10000,
                      valid_sets=[train_data, validation_data],
                      valid_names=["train", "valid"],
                      early_stopping_rounds=100,
                      verbose_eval=100)

        valid_preds = clf.predict(valid_x, num_iteration=clf.best_iteration)
        test_preds = clf.predict(test, num_iteration=clf.best_iteration)

        fold_preds['prob'] = test_preds
        fold_preds['fold_id'] = n_fold + 1
        all_test_preds.append(fold_preds)
        
        oof_preds[valid_idx] = valid_preds
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, valid_preds)))
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    full_auc = roc_auc_score(train_df['label'], oof_preds)
    print('Full AUC score %.6f' % full_auc)    
    
    if not debug:
        train_df["prob"] = oof_preds
        train_df[['id', 'prob']].to_csv(model_dir + "pred_train.csv", index= False)

        all_test_preds = pd.concat(all_test_preds, axis=0)
        all_test_preds.to_csv(model_dir + "all_test_preds.csv", index=False)
        
        sub = pd.DataFrame()
        sub['id'] = all_test_preds.id.unique()
        sub.set_index("id", inplace=True)
        sub["prob"] = all_test_preds.groupby("id").prob.mean()
        sub.reset_index().to_csv(model_dir + "sub.txt", index=False)
    return full_auc

In [96]:
params_list = [
 {'boosting_type': 'goss', 'colsample_bytree': 0.8928229372353017, 'learning_rate': 0.021903389746379412, 'max_bin': 660, 'metric': 'auc', 'min_child_weight': 4.10588643507479, 'num_leaves': 133, 'reg_alpha': 42.97252643187786, 'reg_lambda': 854.733713280976, 'subsample': 1.0, 'seed': 6620},
 {'boosting_type': 'goss', 'colsample_bytree': 0.7652956286647173, 'learning_rate': 0.0150713876857934, 'max_bin': 260, 'metric': 'auc', 'min_child_weight': 14.119780580011295, 'num_leaves': 127, 'reg_alpha': 47.16111776349013, 'reg_lambda': 365.5720140818867, 'subsample': 1.0, 'seed': 7275},
 {'boosting_type': 'goss', 'colsample_bytree': 0.8473968223357727, 'learning_rate': 0.0160176515704046, 'max_bin': 660, 'metric': 'auc', 'min_child_weight': 5.781514346650003, 'num_leaves': 70, 'reg_alpha': 49.08546460105451, 'reg_lambda': 998.0924920004616, 'subsample': 1.0, 'seed': 35},
 {'boosting_type': 'goss', 'colsample_bytree': 0.8101714675581262, 'learning_rate': 0.02180919660536074, 'max_bin': 240, 'metric': 'auc', 'min_child_weight': 17.42104313915958, 'num_leaves': 128, 'reg_alpha': 40.64421480735473, 'reg_lambda': 970.6607731302161, 'subsample': 1.0, 'seed': 8009},
 {'boosting_type': 'goss', 'colsample_bytree': 0.8116865341025844, 'learning_rate': 0.017833779617487368, 'max_bin': 580, 'metric': 'auc', 'min_child_weight': 8.069623451498073, 'num_leaves': 125, 'reg_alpha': 46.03103766146546, 'reg_lambda': 935.6037896087132, 'subsample': 1.0, 'seed': 7475}
]
result = []
for i, params in enumerate(params_list):
    model_dir = "../model_output/random/lgb_585/%d/" % i
    result.append(cv(df, 5, params, model_dir))

(15166, 585) (3793, 585) (6000, 585)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.844317	valid's auc: 0.680233
[200]	train's auc: 0.89963	valid's auc: 0.703215
[300]	train's auc: 0.929268	valid's auc: 0.70731
[400]	train's auc: 0.947881	valid's auc: 0.708258
[500]	train's auc: 0.960939	valid's auc: 0.709184
[600]	train's auc: 0.971886	valid's auc: 0.711018
[700]	train's auc: 0.978359	valid's auc: 0.710667
Early stopping, best iteration is:
[626]	train's auc: 0.974047	valid's auc: 0.711999


/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  1 AUC : 0.711999
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.825732	valid's auc: 0.657443
[200]	train's auc: 0.87024	valid's auc: 0.674768
[300]	train's auc: 0.891148	valid's auc: 0.683661
[400]	train's auc: 0.900402	valid's auc: 0.685586
[500]	train's auc: 0.919678	valid's auc: 0.687849
[600]	train's auc: 0.938966	valid's auc: 0.689224
[700]	train's auc: 0.945865	valid's auc: 0.689427
[800]	train's auc: 0.957835	valid's auc: 0.689959
Early stopping, best iteration is:
[786]	train's auc: 0.955913	valid's auc: 0.691023
Fold  2 AUC : 0.691023
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.847856	valid's auc: 0.650479
[200]	train's auc: 0.897956	valid's auc: 0.670013
[300]	train's auc: 0.925848	valid's auc: 0.677152
[400]	train's auc: 0.946266	valid's auc: 0.682409
[500]	train's auc: 0.959933	valid's auc: 0.682425
Early stopping, best iteration is:
[464]	train's auc: 0.955355	valid's auc: 0.684437
Fold  3 A

/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(15166, 585) (3793, 585) (6000, 585)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.821835	valid's auc: 0.673276
[200]	train's auc: 0.868638	valid's auc: 0.700074
[300]	train's auc: 0.890857	valid's auc: 0.705786
[400]	train's auc: 0.897644	valid's auc: 0.707772
[500]	train's auc: 0.91671	valid's auc: 0.710385
[600]	train's auc: 0.935265	valid's auc: 0.712049
[700]	train's auc: 0.941774	valid's auc: 0.711334
Early stopping, best iteration is:
[651]	train's auc: 0.936915	valid's auc: 0.712635
Fold  1 AUC : 0.712635
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.837575	valid's auc: 0.667268
[200]	train's auc: 0.889519	valid's auc: 0.685583
[300]	train's auc: 0.916218	valid's auc: 0.69502
[400]	train's auc: 0.936047	valid's auc: 0.701252
[500]	train's auc: 0.951128	valid's auc: 0.705348
[600]	train's auc: 0.96196	valid's auc: 0.706838
Early stopping, best iteration is:
[550]	train's auc: 0.956862	valid's auc: 0.7073

[300]	train's auc: 0.904341	valid's auc: 0.67556
[400]	train's auc: 0.923621	valid's auc: 0.680417
[500]	train's auc: 0.939102	valid's auc: 0.682736
[600]	train's auc: 0.95052	valid's auc: 0.683702
[700]	train's auc: 0.959987	valid's auc: 0.685746
[800]	train's auc: 0.967553	valid's auc: 0.683751
Early stopping, best iteration is:
[703]	train's auc: 0.960237	valid's auc: 0.685819
Fold  1 AUC : 0.685819
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.829105	valid's auc: 0.66932
[200]	train's auc: 0.876857	valid's auc: 0.68581
[300]	train's auc: 0.903847	valid's auc: 0.695449
[400]	train's auc: 0.922286	valid's auc: 0.697448
[500]	train's auc: 0.938034	valid's auc: 0.700579
[600]	train's auc: 0.949455	valid's auc: 0.701932
[700]	train's auc: 0.959222	valid's auc: 0.702183
Early stopping, best iteration is:
[639]	train's auc: 0.953646	valid's auc: 0.703734
Fold  2 AUC : 0.703734
Training until validation scores don't improve for 100 rounds.
[100]	train'

In [98]:
result

[0.70231075233575124,
 0.70186254931970571,
 0.70096605695776093,
 0.70082844717154702,
 0.70081768831357527]

In [64]:
# params_list = [
#  {'boosting_type': 'goss', 'colsample_bytree': 0.716607417156733, 'learning_rate': 0.015811886667625046, 'max_bin': 500, 'metric': 'auc', 'min_child_weight': 10.093000922782679, 'num_leaves': 130, 'reg_alpha': 30.944454789678236, 'reg_lambda': 879.1337900040899, 'subsample': 1.0},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.7551913691539023, 'learning_rate': 0.01602856038018904, 'max_bin': 540, 'metric': 'auc', 'min_child_weight': 7.891039000535216, 'num_leaves': 133, 'reg_alpha': 31.4245168486152, 'reg_lambda': 936.9319377239084, 'subsample': 1.0},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.7923762362682664, 'learning_rate': 0.015930856358996392, 'max_bin': 700, 'metric': 'auc', 'min_child_weight': 18.203954954853845, 'num_leaves': 100, 'reg_alpha': 14.731472593656346, 'reg_lambda': 794.8692731261094, 'subsample': 1.0},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.8297704970792237, 'learning_rate': 0.015164958618172615, 'max_bin': 740, 'metric': 'auc', 'min_child_weight': 19.6467664018521, 'num_leaves': 121, 'reg_alpha': 37.542451872400974, 'reg_lambda': 833.0943806882824, 'subsample': 1.0},
#  {'boosting_type': 'goss', 'colsample_bytree': 0.7244912699084813, 'learning_rate': 0.015805821054741424, 'max_bin': 640, 'metric': 'auc', 'min_child_weight': 11.490023875792097, 'num_leaves': 162, 'reg_alpha': 42.76782716751604, 'reg_lambda': 907.1538382801542, 'subsample': 1.0}
# ]
# result = []
# for i, params in enumerate(params_list):
#     model_dir = "../model_output/lgb_585/%d/" % i
#     result.append(cv(df, 5, params, model_dir))

(15166, 585) (3793, 585) (6000, 585)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.840846	valid's auc: 0.665103
[200]	train's auc: 0.886248	valid's auc: 0.682368
[300]	train's auc: 0.914578	valid's auc: 0.686385
[400]	train's auc: 0.936312	valid's auc: 0.690746
[500]	train's auc: 0.952437	valid's auc: 0.693747
[600]	train's auc: 0.963755	valid's auc: 0.693273
Early stopping, best iteration is:
[517]	train's auc: 0.954477	valid's auc: 0.694184


/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold  1 AUC : 0.694184
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.839988	valid's auc: 0.662485
[200]	train's auc: 0.882738	valid's auc: 0.676853
[300]	train's auc: 0.912667	valid's auc: 0.684813
[400]	train's auc: 0.935533	valid's auc: 0.687887
[500]	train's auc: 0.951228	valid's auc: 0.693569
[600]	train's auc: 0.963562	valid's auc: 0.691577
Early stopping, best iteration is:
[565]	train's auc: 0.95939	valid's auc: 0.693783
Fold  2 AUC : 0.693783
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.842529	valid's auc: 0.685922
[200]	train's auc: 0.869256	valid's auc: 0.701892
[300]	train's auc: 0.888639	valid's auc: 0.70628
[400]	train's auc: 0.895631	valid's auc: 0.708702
[500]	train's auc: 0.914765	valid's auc: 0.711725
[600]	train's auc: 0.933269	valid's auc: 0.713695
[700]	train's auc: 0.940264	valid's auc: 0.714737
[800]	train's auc: 0.951556	valid's auc: 0.715198
[900]	train's auc: 0.961631	valid's auc: 0.71

/home/LAB/yanhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(15166, 585) (3793, 585) (6000, 585)
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.840484	valid's auc: 0.664847
[200]	train's auc: 0.88322	valid's auc: 0.680963
[300]	train's auc: 0.912706	valid's auc: 0.690192
[400]	train's auc: 0.935718	valid's auc: 0.693437
[500]	train's auc: 0.950995	valid's auc: 0.695624
[600]	train's auc: 0.963482	valid's auc: 0.698315
[700]	train's auc: 0.972479	valid's auc: 0.701753
[800]	train's auc: 0.979086	valid's auc: 0.702392
Early stopping, best iteration is:
[790]	train's auc: 0.978536	valid's auc: 0.702759
Fold  1 AUC : 0.702759
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.838202	valid's auc: 0.663644
[200]	train's auc: 0.88479	valid's auc: 0.681296
[300]	train's auc: 0.912365	valid's auc: 0.687243
[400]	train's auc: 0.934643	valid's auc: 0.689809
[500]	train's auc: 0.949817	valid's auc: 0.692425
[600]	train's auc: 0.962488	valid's auc: 0.692618
Early stopping, best iteration 

Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.821829	valid's auc: 0.659066
[200]	train's auc: 0.872042	valid's auc: 0.677806
[300]	train's auc: 0.898105	valid's auc: 0.686158
[400]	train's auc: 0.918351	valid's auc: 0.69342
[500]	train's auc: 0.933093	valid's auc: 0.694868
[600]	train's auc: 0.945154	valid's auc: 0.696281
[700]	train's auc: 0.955524	valid's auc: 0.697442
[800]	train's auc: 0.963886	valid's auc: 0.697721
[900]	train's auc: 0.97001	valid's auc: 0.697218
Early stopping, best iteration is:
[815]	train's auc: 0.96492	valid's auc: 0.698161
Fold  1 AUC : 0.698161
Training until validation scores don't improve for 100 rounds.
[100]	train's auc: 0.8261	valid's auc: 0.66364
[200]	train's auc: 0.87096	valid's auc: 0.674709
[300]	train's auc: 0.898377	valid's auc: 0.680149
[400]	train's auc: 0.919059	valid's auc: 0.685643
[500]	train's auc: 0.933441	valid's auc: 0.688964
[600]	train's auc: 0.946041	valid's auc: 0.691102
[700]	train's auc: 0.95